In [1]:
from io import StringIO
import polars as pl
import requests
import kuzu
from ipysigma import Sigma
import networkx as nx
import igraph as ig

In [2]:
def query_mode(source,   query):
    params = {
        "timeout": "600s",
        "access-token": "doos_7643543846_6dMISzlPrD7i"
    }
    headers = {
        "Accept": "text/csv",
        "Content-type": "application/sparql-query"
    }

    # Read the SPARQL query from file
    with open(query, "r") as file:
        query = file.read()

    # Send the request
    response = requests.post(source, params=params, headers=headers, data=query)

    # Load response into Polars DataFrame
    df = pl.read_csv(StringIO(response.text))

    return df

In [3]:
source = "http://0.0.0.0:7019"
query = "/home/fils/scratch/qleverflow/queries/keywordsPathSearch.rq"
df = query_mode(source, query)

In [4]:
combined_uniques = (
        df
        .select([
            pl.concat_list([pl.col('source'), pl.col('target')])
            .alias('nodes')
        ])
        .select([
            pl.col('nodes').explode().unique().sort()
        ])
    ).drop_nulls()    #   .to_series().to_list()

In [5]:
edge_df = df.select(['source', 'target']).drop_nulls()

In [6]:
db = kuzu.Database()
conn = kuzu.Connection(db)

In [7]:
conn.execute("DROP TABLE IF EXISTS Relations")
conn.execute("DROP TABLE IF EXISTS Entity")

conn.execute("CREATE NODE TABLE Entity(nodes STRING, PRIMARY KEY (nodes))")
conn.execute("COPY Entity FROM combined_uniques")
conn.execute("CREATE REL TABLE Relations(FROM Entity TO Entity)")
conn.execute("COPY Relations FROM edge_df")

In [20]:
cq = """MATCH (n1:Entity)-[r]->(n2:Entity)
RETURN n1, n2, r
LIMIT 5000
"""

In [21]:
r = conn.execute(cq)

In [22]:
G = r.get_as_networkx(directed=False)

In [23]:
# sigma can work with a networkx graph to begin with
Sigma(
    G,
    node_size=G.degree,
    default_edge_type='curve',
    node_border_color_from='node',
    node_metrics=['louvain'],
    node_color='louvain',
    start_layout=5,
    edge_size=lambda u, v: G.degree(u) + G.degree(v),
    edge_size_range=(0.5, 5),
    label_font='cursive',
    node_label_size=G.degree,
    label_density=0
)

Sigma(nx.MultiGraph with 2,987 nodes and 5,000 edges)